# Hyper CLOVA 로 테스트해보기 

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

import pickle

In [2]:
import requests
from bs4 import BeautifulSoup as bs


In [3]:
from tqdm import tqdm

In [4]:
CWD = Path.cwd()
WORKSPACE_PATH = CWD.parent
COMMON_PATH = WORKSPACE_PATH / 'common'
DATA_PATH = WORKSPACE_PATH / 'data'

In [22]:
import sys

SEOKBUM_PATH = WORKSPACE_PATH / 'seokbum'

sys.path.append(SEOKBUM_PATH.as_posix())

In [23]:
from completion_executor import ChatCompletionExecutor
from sliding_window_executor import SlidingWindowExecutor
import json

## 데이터 불러오기

- returns
- news articles (full)

In [5]:
returns_df = pd.read_pickle(DATA_PATH / 'returns_df_top100.pkl')

In [6]:
START_DATE = returns_df.index[0].strftime('%Y.%m.%d')
END_DATE = returns_df.index[-1].strftime('%Y.%m.%d')

START_DATE, END_DATE

('2020.01.02', '2024.07.12')

In [7]:
top100_tickers = returns_df.columns.to_list()

In [8]:
news_full = pd.read_pickle(DATA_PATH / 'news_full_00.pkl')

## Graph RAG LLM 적용

In [19]:
def get_txt(title):
    with open(DATA_PATH / f'{title}.txt', 'r', encoding='utf-8') as f:
        return f.read()

In [20]:
한미 = get_txt('한미반도체')

In [21]:
instruction = get_txt('instruction')

In [24]:
sliding_window_executor = SlidingWindowExecutor(
    host='clovastudio.apigw.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY+Uy9rYumYZsV6izDa9AFvO56MDOrkyLi3eCmugWbNRu',
    api_key_primary_val = '1yiWJOyxczG1Ichq95JEUty5cZKr3F5iBynZfQWL',
    request_id='3dd95651-1c46-47e5-b56a-17b24d80669f'
)

completion_executor = ChatCompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY+Uy9rYumYZsV6izDa9AFvO56MDOrkyLi3eCmugWbNRu',
    api_key_primary_val='1yiWJOyxczG1Ichq95JEUty5cZKr3F5iBynZfQWL',
    request_id='c157480f-7a49-4fff-bc49-65227afaaffb'
)

In [25]:
messages = [{"role": "user", "content": instruction}]

message_head = "객체 유형 목록: [\"기업\", \"인물\", \"국가\"]\n\n"
news_title = '뉴스 제목: ' + '한미반도체, 삼성전자 엔비디아 HBM 승인 보도에 한 때 7% 폭락'
news_document = "뉴스 기사 본문: " + "\n\n" + 한미

In [26]:
messages.append({"role": "user", "content": message_head + news_title + news_document})

In [27]:
request_data = {
    "messages": messages,
    "maxTokens": 1024
}

In [28]:
adjusted_messages = sliding_window_executor.execute(request_data)

In [29]:
completion_request_data = {
    "messages": adjusted_messages,
    "maxTokens": 1024, 
    "temperature": 0.5,
    "topK": 0,
    "topP": 0.8,
    "repeatPenalty": 1.2,
    "stopBefore": [],
    "includeAiFilters": True,
    "seed": 0
}

In [30]:
response = completion_executor.execute(completion_request_data)

In [31]:
extract_result = response.get('result', {}).get('message', {}).get('content', '').strip()

In [32]:
print(extract_result)

출력:

- ("객체", "한미반도체", "기업", "한미반도체는 반도체 장비 대장주로, 삼성전자와 SK하이닉스에 장비를 납품한다.")
- ("객체", "삼성전자", "기업", "삼성전자는 대한민국의 대표적인 IT 기업으로, 반도체, 스마트폰, 가전제품 등을 생산한다.")
- ("객체", "엔비디아", "기업", "엔비디아는 미국의 그래픽 처리 장치(GPU) 제조 기업으로, 인공지능(AI) 분야에서도 높은 기술력을 보유하고 있다.")
- ("객체", "곽동신", "인물", "곽동신은 한미반도체의 부회장으로, 한미반도체의 최대주주이다.")
- ("관계", "한미반도체", "삼성전자", "한미반도체는 삼성전자에 HBM 제조용 TC본더를 납품하지 않고 있으며, 삼성전자가 엔비디아의 HBM 납품 퀄테스트를 통과하면서 한미반도체의 납품 가능성이 낮아졌다.", -2)
- ("관계", "삼성전자", "엔비디아", "삼성전자는 엔비디아의 HBM 납품 퀄테스트를 통과했으며, 공식적인 HBM 공급을 위한 양산에 본격 나설 것으로 보인다.", 5)
- ("관계", "한미반도체", "SK하이닉스", "한미반도체는 SK하이닉스의 주력 납품사로 미국 마이크론테크놀로지까지 고객사로 확보했다", 3)
